#RASPADO DE COMPONENTES HISTÓRICOS S&P 500

In [1]:
# Importar las librerías necesarias
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime

# Definir la URL de la página de Wikipedia para el raspado
URL = "https://en.wikipedia.org/wiki/List_of_S%26P_500_companies"

# Función para realizar la solicitud HTTP y obtener el contenido
def get_html_content(url):
    response = requests.get(url)
    # Confirmar que la solicitud fue exitosa (código de estado HTTP 200)
    if response.status_code == 200:
        return response.text
    else:
        raise Exception(f"Error al obtener el contenido de la página, código de estado: {response.status_code}")

# Función para parsear el contenido HTML con BeautifulSoup
def parse_html_to_soup(html_content):
    soup = BeautifulSoup(html_content, 'html.parser')
    # Confirmar que se creó el objeto BeautifulSoup
    return soup

# Obtener el contenido HTML de la página de Wikipedia
html_content = get_html_content(URL)

# Parsear el contenido HTML para obtener el objeto BeautifulSoup
soup = parse_html_to_soup(html_content)

# Verificar que se extrajo contenido significativo
# Para ello, buscar un elemento distintivo de la página que sabemos que debería estar presente
test_element = soup.find('h1', id="firstHeading")
assert test_element is not None, "El contenido HTML no contiene el elemento esperado."

print("El contenido HTML ha sido extraído y parseado correctamente.")


El contenido HTML ha sido extraído y parseado correctamente.


In [2]:
# Función para extraer los datos de la tabla de componentes actuales del S&P 500
def extract_current_snp_data(soup):
    table = soup.find('table', {'id': 'constituents'})
    rows = table.find_all('tr')[1:]  # Excluir el encabezado de la tabla

    current_data = []
    for row in rows:
        cols = row.find_all('td')
        # Asumimos que la tabla tiene el formato correcto y que no hay columnas faltantes
        security = cols[1].text.strip()
        symbol = cols[0].text.strip()
        current_data.append({
            'Security': security,
            'Symbol': symbol,
            'Date': '1900-01-01'  # Fecha fija para indicar que son miembros originales
        })

    return current_data

def extract_historical_changes_data(soup):
    table = soup.find('table', {'id': 'changes'})
    rows = table.find_all('tr')[1:]  # Excluir el encabezado de la tabla

    historical_data = []
    for i, row in enumerate(rows):
        cols = row.find_all('td')
        if len(cols) < 6:  # Asegurarse de que hay suficientes columnas
            print(f"Fila {i+1} omitida: no contiene suficientes columnas.")
            continue

        # Parsear la fecha y asegurarse de que tenga el formato correcto
        date_text = cols[0].text.strip()
        try:
            date = datetime.strptime(date_text, '%B %d, %Y').strftime('%Y-%m-%d')
        except ValueError:
            print(f"Error al parsear la fecha en la fila {i+1}: {date_text}")
            continue

        # Extraer símbolos y seguridades agregadas y eliminadas
        added_symbol = cols[1].text.strip()
        added_security = cols[2].text.strip()
        removed_symbol = cols[3].text.strip()
        removed_security = cols[4].text.strip()

        # Agregar al DataFrame los valores agregados y eliminados como entradas separadas
        historical_data.append({'Security': added_security, 'Symbol': added_symbol, 'Date': date, 'Change': 'Added'})
        historical_data.append({'Security': removed_security, 'Symbol': removed_symbol, 'Date': date, 'Change': 'Removed'})

    return historical_data


# Extraer datos y crear DataFrames
current_data = extract_current_snp_data(soup)
historical_data = extract_historical_changes_data(soup)

dataframe_current = pd.DataFrame(current_data)
dataframe_historical = pd.DataFrame(historical_data)

# Mostrar los primeros registros para confirmar
print(dataframe_current.head())
print(dataframe_historical.head())


Fila 1 omitida: no contiene suficientes columnas.
      Security Symbol        Date
0           3M    MMM  1900-01-01
1  A. O. Smith    AOS  1900-01-01
2       Abbott    ABT  1900-01-01
3       AbbVie   ABBV  1900-01-01
4    Accenture    ACN  1900-01-01
              Security Symbol        Date   Change
0              Hubbell   HUBB  2023-10-18    Added
1        Organon & Co.    OGN  2023-10-18  Removed
2  Lululemon Athletica   LULU  2023-10-18    Added
3  Activision Blizzard   ATVI  2023-10-18  Removed
4                              2023-10-03    Added


## Datos faltantes

In [8]:
# Revisar datos faltantes en la columna 'Symbol' para el DataFrame de componentes actuales
missing_symbols_current = dataframe_current['Symbol'].isnull().sum()
print("Datos faltantes en la columna 'Symbol' para los componentes actuales del S&P 500:")
print(missing_symbols_current)
print("\nFilas con datos faltantes en la columna 'Symbol' para los componentes actuales del S&P 500:")
print(dataframe_current[dataframe_current['Symbol'].isnull()])

# Revisar datos faltantes en la columna 'Symbol' para el DataFrame de cambios históricos
missing_symbols_historical = dataframe_historical['Symbol'].isnull().sum()
print("\nDatos faltantes en la columna 'Symbol' para los cambios históricos del S&P 500:")
print(missing_symbols_historical)
print("\nFilas con datos faltantes en la columna 'Symbol' para los cambios históricos del S&P 500:")
print(dataframe_historical[dataframe_historical['Symbol'].isnull()])

# Identificar símbolos que son cadenas vacías o solo espacios en el DataFrame de componentes actuales
empty_symbols_current = dataframe_current[dataframe_current['Symbol'].str.strip() == '']
print("Símbolos vacíos en los componentes actuales del S&P 500:")
print(empty_symbols_current)

# Identificar símbolos que son cadenas vacías o solo espacios en el DataFrame de cambios históricos
empty_symbols_historical = dataframe_historical[dataframe_historical['Symbol'].str.strip() == '']
print("\nSímbolos vacíos en los cambios históricos del S&P 500:")
print(empty_symbols_historical)



Datos faltantes en la columna 'Symbol' para los componentes actuales del S&P 500:
0

Filas con datos faltantes en la columna 'Symbol' para los componentes actuales del S&P 500:
Empty DataFrame
Columns: [Security, Symbol, Date]
Index: []

Datos faltantes en la columna 'Symbol' para los cambios históricos del S&P 500:
0

Filas con datos faltantes en la columna 'Symbol' para los cambios históricos del S&P 500:
Empty DataFrame
Columns: [Security, Symbol, Date, Change]
Index: []
Símbolos vacíos en los componentes actuales del S&P 500:
Empty DataFrame
Columns: [Security, Symbol, Date]
Index: []

Símbolos vacíos en los cambios históricos del S&P 500:
    Security Symbol        Date   Change
4                    2023-10-03    Added
7                    2023-10-02  Removed
24                   2023-01-05    Added
27                   2023-01-04  Removed
32                   2022-12-19    Added
35                   2022-12-15  Removed
48                   2022-06-21    Added
62                  

In [6]:
# Eliminar filas donde la columna 'Symbol' contiene una cadena vacía o solo espacios en blanco
dataframe_historical_cleaned = dataframe_historical[dataframe_historical['Symbol'].str.strip() != '']

# Verificar que las filas han sido eliminadas
print(dataframe_historical_cleaned)


                          Security Symbol        Date   Change
0                          Hubbell   HUBB  2023-10-18    Added
1                    Organon & Co.    OGN  2023-10-18  Removed
2              Lululemon Athletica   LULU  2023-10-18    Added
3              Activision Blizzard   ATVI  2023-10-18  Removed
5                   DXC Technology    DXC  2023-10-03  Removed
..                             ...    ...         ...      ...
663                     General Re    GRN  1998-12-11  Removed
664                      Compuware   CPWR  1998-12-11    Added
665                     SunAmerica    SUN  1998-12-11  Removed
666  Countrywide Credit Industries    CCI  1997-06-17    Added
667                         USLife    USL  1997-06-17  Removed

[641 rows x 4 columns]


In [7]:

# Identificar símbolos que son cadenas vacías o solo espacios en el DataFrame de cambios históricos
empty_symbols_historical = dataframe_historical_cleaned[dataframe_historical_cleaned['Symbol'].str.strip() == '']
print("\nSímbolos vacíos en los cambios históricos del S&P 500:")
print(empty_symbols_historical)



Símbolos vacíos en los cambios históricos del S&P 500:
Empty DataFrame
Columns: [Security, Symbol, Date, Change]
Index: []


In [9]:
# Guardar el DataFrame de componentes actuales del S&P 500 en un archivo CSV
dataframe_current.to_csv('current_snp500.csv', index=False)

# Guardar el DataFrame de cambios históricos del S&P 500 limpio en un archivo CSV
dataframe_historical_cleaned.to_csv('historical_changes_snp500.csv', index=False)



Los archivos han sido guardados.


## Crear base de consulta (Si el stock pertenecía o no al S&P)

In [51]:

# Paso 1: Combinar y Preparar DataFrames
def preparar_dataframes(dataframe_current, dataframe_historical):
    fecha_inicio = pd.to_datetime('1900-01-01')
    dataframe_current['Date'] = fecha_inicio
    dataframe_current['Change'] = 'Added'

    # Crear una copia de los datos añadidos del dataframe histórico
    added_historical = dataframe_historical[dataframe_historical['Change'] == 'Added'].drop_duplicates(subset='Symbol', keep='first')

    # Actualizar el dataframe current con los datos de añadido del historical
    for index, row in added_historical.iterrows():
        current_index = dataframe_current[dataframe_current['Symbol'] == row['Symbol']].index
        if not current_index.empty:
            dataframe_current.loc[current_index, 'Date'] = row['Date']

    dataframe_combinado = pd.concat([dataframe_current, dataframe_historical]).drop_duplicates(subset=['Symbol', 'Date', 'Change'], keep='last')
    dataframe_combinado.sort_values(by=['Symbol', 'Date'], inplace=True)

    return dataframe_combinado, added_historical

# Paso 2: Crear Diccionario de Estados de Stocks
def crear_diccionario_estados(dataframe_combinado, added_historical):
    estados = {}
    fecha_inicio = pd.to_datetime('1900-01-01')
    for _, row in dataframe_combinado.iterrows():
        if row['Symbol'] not in estados:
            estados[row['Symbol']] = []
            # Si el stock fue eliminado pero no tiene fecha de adición, asignarle la fecha de inicio
            if row['Change'] == 'Removed' and row['Symbol'] not in added_historical['Symbol'].values:
                estados[row['Symbol']].append((fecha_inicio, 'Added'))
        estados[row['Symbol']].append((row['Date'], row['Change']))
    return estados

# Paso 3: Función de Verificación de Pertenencia
def verificar_pertenencia(símbolo, fecha, estados):
    fecha = pd.to_datetime(fecha)  # Asegurarse de que 'fecha' sea un Timestamp
    cambios = estados.get(símbolo, [])

    pertenecía = False
    for cambio_fecha, cambio_estado in sorted(cambios, key=lambda x: x[0]):
        if cambio_fecha > fecha:
            break
        pertenecía = cambio_estado == 'Added'
    return pertenecía


# Paso 4: Función para Procesar Consultas Múltiples
def procesar_consultas(consultas, estados):
    resultados = []
    for símbolo, fecha in consultas:
        pertenece = verificar_pertenencia(símbolo, fecha, estados)
        resultados.append((símbolo, fecha, pertenece))
    return resultados

# Preparar DataFrames y crear el diccionario de estados
dataframe_combinado, added_historical = preparar_dataframes(dataframe_current, dataframe_historical)
estados = crear_diccionario_estados(dataframe_combinado, added_historical)

# Ejemplo de uso
consultas = [("HRL", "2007-01-01"), ("HRL", "2009-01-01")]
resultados = procesar_consultas(consultas, estados)
for símbolo, fecha, pertenece in resultados:
    print(f"El símbolo {símbolo} {'pertenecía' if pertenece else 'no pertenecía'} al S&P 500 en la fecha {fecha}.")

El símbolo HRL no pertenecía al S&P 500 en la fecha 2007-01-01.
El símbolo HRL no pertenecía al S&P 500 en la fecha 2009-01-01.


In [35]:
dataframe_combinado.to_csv('dataframe_combinado.csv', index=False)
print("Los archivos han sido guardados.")

Los archivos han sido guardados.


### Intento 1

In [34]:
# Bloque 1: Combinar y ordenar los DataFrames
def combine_and_sort_dataframes(current_df, historical_df):
    # Concatena con el DataFrame histórico
    combined_df = pd.concat([current_df, historical_df], ignore_index=True)

    # Eliminar duplicados manteniendo la última entrada (que debería ser la más reciente si hay duplicados)
    combined_df.drop_duplicates(subset=['Symbol'], keep='last', inplace=True)

    # Ordenar por 'Date' en orden ascendente y luego por 'Symbol' en orden alfabético
    combined_df.sort_values(by=['Date', 'Symbol'], ascending=[True, True], inplace=True)

    return combined_df

# Aplica la función y combina los DataFrames
combined_snp_data = combine_and_sort_dataframes(dataframe_current, dataframe_historical_cleaned)

# Verificar la combinación y el orden de los DataFrames
print(combined_snp_data.head())

# Guardar el DataFrame combinado en un archivo CSV
combined_snp_data.to_csv('combined_snp_data.csv', index=False)

print("El archivo 'combined_snp_data.csv' ha sido guardado.")


                Security Symbol        Date Change
10  Agilent Technologies      A  1900-01-01    NaN
42            Apple Inc.   AAPL  1900-01-01    NaN
2                 Abbott    ABT  1900-01-01    NaN
6             Adobe Inc.   ADBE  1900-01-01    NaN
38        Analog Devices    ADI  1900-01-01    NaN
El archivo 'combined_snp_data.csv' ha sido guardado.


In [36]:
# Subpaso 1: Crear un diccionario para mantener el registro temporal de cada stock
stock_history = {}

# Subpaso 2: Llenar el diccionario con la información del DataFrame
for index, row in combined_snp_data.iterrows():
    symbol = row['Symbol']
    change_date = pd.to_datetime(row['Date'])
    change_type = row['Change']

    if symbol not in stock_history:
        stock_history[symbol] = []

    stock_history[symbol].append((change_date, change_type))

# Ordenar las listas de tuplas por fecha para cada símbolo
for symbol in stock_history:
    stock_history[symbol].sort(key=lambda x: x[0])

# Verificar una parte del diccionario para confirmar
for symbol, changes in list(stock_history.items())[:5]:  # Solo imprimir los primeros 5 para verificación
    print(f"{symbol}: {changes}")


A: [(Timestamp('1900-01-01 00:00:00'), nan)]
AAPL: [(Timestamp('1900-01-01 00:00:00'), nan)]
ABT: [(Timestamp('1900-01-01 00:00:00'), nan)]
ADBE: [(Timestamp('1900-01-01 00:00:00'), nan)]
ADI: [(Timestamp('1900-01-01 00:00:00'), nan)]


In [82]:
import pandas as pd
import csv

# Asegurarse de que todos los stocks actuales tienen una entrada de 'Added'
for symbol in dataframe_current['Symbol']:
    if symbol not in stock_history or not any(change_type == 'Added' for _, change_type in stock_history[symbol]):
        stock_history[symbol] = [(pd.Timestamp('1900-01-01'), 'Added')]

# Reemplazar 'nan' con 'Added' en el diccionario y escribir el diccionario corregido en un archivo CSV
with open('stock_history_corrected.csv', 'w', newline='') as csvfile:
    fieldnames = ['Symbol', 'Change Date', 'Change Type']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for symbol, changes in stock_history.items():
        for change_date, change_type in changes:
            # Convertir la fecha en string si es un objeto Timestamp
            change_date_str = change_date.strftime('%Y-%m-%d') if not isinstance(change_date, str) else change_date
            writer.writerow({'Symbol': symbol, 'Change Date': change_date_str, 'Change Type': change_type})

print("El diccionario se ha guardado en 'stock_history_corrected.csv'.")



El diccionario se ha guardado en 'stock_history_corrected.csv'.


In [81]:
def is_stock_in_snp_on_date(symbol, date, history_dict):
    # Convertir la fecha dada a un objeto Timestamp para comparación
    date = pd.to_datetime(date)
    # Verificar si el símbolo existe en el diccionario de historial
    if symbol not in history_dict:
        return False

    # Obtener la historia del símbolo
    symbol_history = history_dict[symbol]

    # Inicializar la variable para mantener el estado actual del símbolo
    current_status = False  # Asumir que no está en el S&P 500 hasta encontrar lo contrario

    # Revisar la historia del símbolo hasta la fecha dada
    for change_date, status in symbol_history:
        if change_date > date:
            break  # Todos los cambios después de la fecha dada no son relevantes
        if status == 'Added':
            current_status = True
        elif status == 'Removed':
            current_status = False

    return current_status

# Ejemplo de uso de la función:
# Verificar si 'Symbol' estaba en el S&P 500 en una fecha específica
was_aapl_in_snp = is_stock_in_snp_on_date('HUBB', '2023-10-17', stock_history)
print(f"Was the stock in S&P 500 on the date? {was_aapl_in_snp}")


Debug 2

In [83]:
def is_stock_in_snp_on_date(symbol, query_date, history_dict):
    query_date = pd.to_datetime(query_date)
    if symbol not in history_dict:
        return False
    is_in_index = False
    was_ever_added = any(status == 'Added' for change_date, status in history_dict[symbol])
    for change_date, status in sorted(history_dict[symbol], key=lambda x: x[0]):
        if change_date <= query_date:
            is_in_index = status == 'Added'
        elif change_date > query_date and was_ever_added:
            break
    if not was_ever_added and is_in_index is False:
        is_in_index = True
    return is_in_index

# Ejemplo de uso de la función para un símbolo que ha sido eliminado
was_symbol_in_snp = is_stock_in_snp_on_date('USL', '1997-06-16', stock_history)  # Debe ser True, día antes de la eliminación
print(f"Was USL in S&P 500 on 1997-06-16? {was_symbol_in_snp}")

was_symbol_in_snp = is_stock_in_snp_on_date('USL', '1997-06-18', stock_history)  # Debe ser False, día después de la eliminación
print(f"Was USL in S&P 500 on 1997-06-18? {was_symbol_in_snp}")



Was USL in S&P 500 on 1997-06-16? True
Was USL in S&P 500 on 1997-06-18? True


###Debug

----------

-----